## 读取数据集

In [5]:
import os
os.makedirs(os.path.join('..','data'), exist_ok=True)
datafile = os.path.join('..','data','house_tiny.csv')

In [6]:

with open(datafile, 'w') as f:
    f.write('NumRooms,Alley,Price\n') # 列名
    f.write('NA,Pave,127500\n') # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [7]:
import pandas as pd

In [8]:
data = pd.read_csv(datafile)
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


## 处理缺失值

In [21]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs,outputs

(   NumRooms Alley
 0       NaN  Pave
 1       2.0   NaN
 2       4.0   NaN
 3       NaN   NaN,
 0    127500
 1    106000
 2    178100
 3    140000
 Name: Price, dtype: int64)

In [22]:
import numpy as np

# 找到所有的数值列
numeric_cols = inputs.select_dtypes(include=np.number).columns
numeric_cols

Index(['NumRooms'], dtype='object')

In [24]:
inputs[numeric_cols] = inputs[numeric_cols].fillna(inputs[numeric_cols].mean())
inputs

,NumRooms,Alley
0,3.0,Pave
1,2.0,NaN
2,4.0,NaN
3,3.0,NaN


`pd.get_dummies()` 的工作方式非常直观。它会：

- 找到指定列中的所有唯一类别。
- 为每一个类别创建一个新的列。
- 如果原始数据中的某一行属于这个类别，那么在新列中对应行的值就为 1，否则为 0。 (新版本中`dtype=bool`是默认行为)

In [25]:
inputs = pd.get_dummies(inputs,dummy_na=True,dtype='int')
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


## 转换为张量格式

In [27]:
import torch
X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(outputs.to_numpy(dtype=float))
X,y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

## 删除缺失值最多的单元

In [28]:
import pandas as pd
import numpy as np

# 1. 创建数据
# 我们用一个字典来创建 DataFrame，其中 np.nan 代表缺失值。
data = {
    '学生ID': [101, 102, 103, 104, 105, 106, 107],
    '性别': ['男', '女', '女', '男', '男', np.nan, '女'], # 缺失 1 个
    '年龄': [18, 19, 18, 20, 19, 19, 18],              # 缺失 0 个
    '成绩': [85, 92, 78, np.nan, 88, 95, np.nan],       # 缺失 2 个
    '兴趣爱好': ['篮球', np.nan, '阅读', np.nan, np.nan, '音乐', np.nan] # 缺失 4 个
}

df = pd.DataFrame(data)

print("--- 步骤 1: 原始的 DataFrame ---")
print(df)

--- 步骤 1: 原始的 DataFrame ---
   学生ID   性别  年龄    成绩 兴趣爱好
0   101    男  18  85.0   篮球
1   102    女  19  92.0  NaN
2   103    女  18  78.0   阅读
3   104    男  20   NaN  NaN
4   105    男  19  88.0  NaN
5   106  NaN  19  95.0   音乐
6   107    女  18   NaN  NaN


In [29]:
# 2. 计算每列的缺失值总数
missing_counts = df.isnull().sum()

print("\n--- 步骤 2: 每列的缺失值数量 ---")
print(missing_counts)


--- 步骤 2: 每列的缺失值数量 ---
学生ID    0
性别      1
年龄      0
成绩      2
兴趣爱好    4
dtype: int64


In [30]:
# 3. 找到缺失值最多的列的名称
col_to_drop = missing_counts.idxmax()

print("\n--- 步骤 3: 准备删除的列 ---")
print(f"缺失值最多的列是: '{col_to_drop}'，共有 {missing_counts.max()} 个缺失值。")


--- 步骤 3: 准备删除的列 ---
缺失值最多的列是: '兴趣爱好'，共有 4 个缺失值。


In [31]:
# 4. 删除找到的列
df_cleaned = df.drop(columns=[col_to_drop])

print("\n--- 步骤 4: 删除后的 DataFrame ---")
print(df_cleaned)


--- 步骤 4: 删除后的 DataFrame ---
   学生ID   性别  年龄    成绩
0   101    男  18  85.0
1   102    女  19  92.0
2   103    女  18  78.0
3   104    男  20   NaN
4   105    男  19  88.0
5   106  NaN  19  95.0
6   107    女  18   NaN
